# Experimenting with HuggingFace - Text Generation

*Author: Jolene Chen*

**I have recently decided to explore the ins and outs of the 😊 Transformers library and this is the next chapter in that journey. In this notebook, I will explore text generation using a GPT-2 model, which was trained to predict next words on 40GB of Internet text data. The fully trained model is actually not available as the creators were concerned about '[malicious applications of the technology](https://openai.com/blog/better-language-models/)', but there is a much smaller version that is available for enthusiants to play with, which we will use here**

**In this notebook, we will explore different decoding methods like Beam search, Top-K sampling, and Top-P sampling, demonstrating their performance along the way. This project is a work in progress and I will continually update it as I learn more about text generation. Feel free to comment with any questions/suggestions:**

**Update: they just released the GPT-3 model (more [here](https://www.technologyreview.com/2020/07/20/1005454/openai-machine-learning-language-generator-gpt-3-nlp/)) and it has 175 billion parameters and it is shockingly intelligent**

In [1]:
#for reproducability
import random
SEED = 42
random.seed(SEED)

#maximum number of words in output text
MAX_LEN = 70

In [2]:
input_sequence = "I don't know about you, but there's only one thing I want to do after a long day of work"

# I. Intro

**A language model is a machine learning model that can look at part of a sentence and predict the next word/sequence of words. Much like the autofill features on your iPhone/Android, GPT-2 is capable of next word prediction on a much larger and more sophisticated scale. For reference, the smallest available GPT-2 has 117 million parameters, whereas the largest one (invisible to the public) has over 1.5 billion parameters. The largest one available for public use is half the size of their main GPT-2 model**

**😊 Transformers makes it very easy to import this model with both PyTorch and TensorFlow - in this notebook we will be using TensorFlow but it is just as easy in PyTorch. Both the model and its Tokenizer can be imported from the `transformers` library that anyone can get by typing `!pip install transformers`. Let's see just how simple it is to generate text with a neural network. We begin with our input sequence:**

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

#get large GPT2 tokenizer and GPT2 model
tokenizer = AutoTokenizer.from_pretrained("gpt2-large")
GPT2 = AutoModelForCausalLM.from_pretrained("gpt2-large", pad_token_id=tokenizer.eos_token_id)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

2025-08-19 01:17:14.434399: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755566234.628055      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755566234.684540      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
#view model parameters
from torchinfo import summary
summary(GPT2)

Layer (type:depth-idx)                             Param #
GPT2LMHeadModel                                    --
├─GPT2Model: 1-1                                   --
│    └─Embedding: 2-1                              64,328,960
│    └─Embedding: 2-2                              1,310,720
│    └─Dropout: 2-3                                --
│    └─ModuleList: 2-4                             --
│    │    └─GPT2Block: 3-1                         19,677,440
│    │    └─GPT2Block: 3-2                         19,677,440
│    │    └─GPT2Block: 3-3                         19,677,440
│    │    └─GPT2Block: 3-4                         19,677,440
│    │    └─GPT2Block: 3-5                         19,677,440
│    │    └─GPT2Block: 3-6                         19,677,440
│    │    └─GPT2Block: 3-7                         19,677,440
│    │    └─GPT2Block: 3-8                         19,677,440
│    │    └─GPT2Block: 3-9                         19,677,440
│    │    └─GPT2Block: 3-10                 

In [5]:
GPT2.config

GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1280,
  "n_head": 20,
  "n_inner": null,
  "n_layer": 36,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.52.4",
  "use_cache": true,
  "vocab_size": 50257
}

# II. Different Decoding Methods

## First Pass (Greedy Search)

**With Greedy search, the word with the highest probability is predicted as the next word i.e. the next word is updated via:**

$$w_t = argmax_{w}P(w | w_{1:t-1})$$

**at each timestep $t$. Let's see how this naive approach performs:**

In [6]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode(input_sequence, return_tensors='pt')

# generate text until the output length (which includes the context length) reaches 50
greedy = GPT2.generate(input_ids, max_length=MAX_LEN)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy[0], skip_special_tokens = True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Output:
----------------------------------------------------------------------------------------------------
I don't know about you, but there's only one thing I want to do after a long day of work: go to the gym.

I'm not talking about the gym that's right next to my house. I'm talking about the gym that's right next to my office.

I'm not talking about the gym that


And there we go: generating text is that easy. Our results are not great - as we can see, our model starts repeating itself rather quickly. The main issue with Greedy Search is that words with high probabilities can be masked by words in front of them with low probabilities, so the model is unable to explore more diverse combinations of words. We can prevent this by implementing Beam Search:

## Beam Search with N-Gram Penalities

**Beam search is essentially Greedy Search but the model tracks and keeps `num_beams` of hypotheses at each time step, so the model is able to compare alternative paths as it generates text. We can also include a n-gram penalty by setting `no_repeat_ngram_size = 2` which ensures that no 2-grams appear twice. We will also set `num_return_sequences = 5` so we can see what the other 5 beams looked like**

**To use Beam Search, we need only modify some parameters in the `generate` function:**

In [7]:
# set return_num_sequences > 1
beam_outputs = GPT2.generate(
    input_ids, 
    max_length = MAX_LEN, 
    num_beams = 5, 
    no_repeat_ngram_size = 2, 
    num_return_sequences = 5, 
    early_stopping = True
)

print('')
print("Output:\n" + 100 * '-')

# now we have 3 output sequences
for i, beam_output in enumerate(beam_outputs):
      print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Output:
----------------------------------------------------------------------------------------------------
0: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to sit down and watch a movie."

"I know, I know," you say. "But you're not going to like this one. It's not a good movie. I mean, it's
1: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to sit down and watch a movie."

"I know, I know," you say. "But you're not going to like this one. It's about a guy who has a crush on a girl
2: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to sit down and watch a movie."

"I know, I know," you say. "But you're not going to like this one. It's about a guy who has a crush on a woman
3: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to sit down and watch a movie."

"I know, I know," 

**Now that's much better! The 5 different beam hypotheses are pretty much all the same, but if we increaed `num_beams`, then we would see some more variation in the separate beams. But of course, Beam Search is not perfect either. It works well when the legnth of the generated text is more or less constant, like problems in translation or summarization, but not so much for open-ended problems like dialog or story generation (because it is much harder to find a balance between `num_beams` and `no_repeat_ngram_size`)**

**Furthermore, [research](https://arxiv.org/abs/1904.09751) shows that human languages do not follow this 'high probability word next' distribution. This makes sense - if my words were exactly what you expected them to be, I would be quite a boring person and most people don't want to be boring! The below graph plots the difference of Beam Search and actual human speech: ![alt text](https://blog.fastforwardlabs.com/images/2019/05/Screen_Shot_2019_05_08_at_3_06_36_PM-1557342561886.png)**

Taken from original paper [here](https://arxiv.org/abs/1904.09751)

## Basic Sampling

**Now we will explore indeterministic decodings - sampling. Instead of following a strict path to find the end text with the highest probability, we instead randomly pick the next word by its conditional probability distribution:**

$$w_t \sim P(w|w_{1:t-1})$$

**However, when we include this randomness, the generated text tends to be incoherent (see more [here](https://arxiv.org/pdf/1904.09751.pdf)) so we can include the `temperature` parameter which increases the chances of high probability words and decreases the chances of low probability words in the sampling:**

**We just need to set `do_sample = True` to implement sampling and for demonstration purposes (you'll shortly see why) we set `top_k = 0`:**

In [8]:
# use temperature to decrease the sensitivity to low probability candidates
sample_output = GPT2.generate(
                             input_ids, 
                             do_sample = True, 
                             max_length = MAX_LEN, 
                             top_k = 0, 
                             temperature = 0.8
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I don't know about you, but there's only one thing I want to do after a long day of work: I want to stop and take a walk."


## Top-K Sampling

**In Top-K sampling, the top k most likely next words are selected and the entire probability mass is shifted to these k words. So instead of increasing the chances of high probability words occuring and decreasing the chances of low probabillity words, we just remove low probability words all together**

**We just need to set `top_k` to however many of the top words we want to consider for our conditional probability distribution:**

In [9]:
#sample from only top_k most likely words
sample_output = GPT2.generate(
                             input_ids, 
                             do_sample = True, 
                             max_length = MAX_LEN, 
                             top_k = 50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True), '...')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I don't know about you, but there's only one thing I want to do after a long day of work and after I've finished up an interview. I want to lie down and sleep. And I want you to take me there. I want you to lay down with me just that one time. You know, after all, that's ...


Top-K Sampling seems to generate more coherent text than our random sampling before. But we can do even better:


## Top-P Sampling

**Top-P sampling (also known as nucleus sampling) is similar to Top-K, but instead of choosing the top k most likely wordsm we choose the smallest set of words whose total probability is larger than p, and then the entire probability mass is shifted to the words in this set**

**The main difference here is that with Top-K sampling, the size of the set of words is static (obviously) whereas in Top-P sampling, the size of the set can change. To use this sampling method, we just set `top_k = 0` and choose a value `top_p`, where 0 <`top_p`< 1:**

In [10]:
#sample only from 80% most likely words
sample_output = GPT2.generate(
                             input_ids, 
                             do_sample = True, 
                             max_length = MAX_LEN, 
                             top_p = 0.8, 
                             top_k = 0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True), '...')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I don't know about you, but there's only one thing I want to do after a long day of work. I want to do a little chat. If you need help, you can call, or send me a message here:

nacho965@gmail.com

The Question

Who are the worst offensive linemen ...


## Top-K and Top-P Sampling

**As you could have probably guessed, we can use both Top-K and Top-P sampling here. This reduces the chances of us getting weird words (low probability words) while allowing for a dynamic selection size. We need only top a value for both `top_k` and `top_p`. We can even include the inital temperature parameter if we want to, Let's now see how our model performs now after adding everything together. We will check the top 5 return to see how diverse our answers are:**

In [11]:
#combine both sampling techniques
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = 2*MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .7,
                              top_k = 50, 
                              top_p = 0.85, 
                              num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0: I don't know about you, but there's only one thing I want to do after a long day of work: sit down and read a book."

The first thing you'll notice is the book is not a traditional biography. Instead, this book takes readers into the author's private world, through his experiences in the military. The story starts at the very beginning and follows the journey of two soldiers—one on active duty and one on reserve—who both have similar experiences. Their lives and choices diverge, but in the end, they come to terms with their experiences and are able to move forward.

"I'm not a great reader," says the author. "When...

1: I don't know about you, but there's only one thing I want to do after a long day of work: make sure everyone is happy!

I don't know if I'll be able to get the photos I wanted for this post, but hopefully this will at least give you some ideas on what to capt

# III. Benchmark Prompts

**Here, we will see how well the GPT-2 model does when given some more interesting inputs. The following prompts are taken from [OpenAI's GPT2](https://openai.com/blog/better-language-models/) website where they feed them to their full sized GPT2 model (and the results are astounding, I highly recommend you check out their [page](https://openai.com/)**

## "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English."

In [18]:
prompt1 = 'In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.'

input_ids = tokenizer.encode(prompt1, return_tensors='pt')

In [19]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50, 
                              top_p = 0.85 
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0: In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

They were found in the wild and had been living in the remote valley for more than a century. However, this...



## "Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today."

**Can we use GPT-2 to generate fake news stories?**

In [16]:
prompt2 = 'Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today.'

input_ids = tokenizer.encode(prompt2, return_tensors='pt')

In [17]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50, 
                              top_p = 0.85
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0: Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today.

The singer was spotted wearing a dress she bought from the department store, which has been widely mocked for the price tag.

Abercrombie and Fitch, which is owned by department store giant Gap Inc, is known for...



## "Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry."

**Can we use GPT-2 to imagine alternate histories of Lord of the Rings?**

In [20]:
prompt3 = 'Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry.'

input_ids = tokenizer.encode(prompt3, return_tensors='pt')

In [21]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50, 
                              top_p = 0.85 
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0: Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry. They could only watch as the great army came marching towards them. A battle was raging in the sky, the orcs were battling and the horses were trampling the soldiers to the ground. Gimli and Aragorn had fought well in the past and they...



## "For today’s homework assignment, please describe the reasons for the US Civil War."

**Can we use GPT-2 to do our homework?**

In [22]:
prompt4 = "For today’s homework assignment, please describe the reasons for the US Civil War."

input_ids = tokenizer.encode(prompt4, return_tensors='pt')

In [23]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50, 
                              top_p = 0.85 
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0: For today’s homework assignment, please describe the reasons for the US Civil War. If the US was attacked or threatened with being attacked, would you defend the US or not? Would you support the US or not? The following questions are simple, but are likely to provoke some interesting conversations among your peers. Your answer may be used to help...

